In [18]:
%pip install git+https://github.com/MarcusLoppe/meshgpt-pytorch.git
%pip install matplotlib
from pathlib import Path 
import gc    
import torch
import os
import torch  
from meshgpt_pytorch import (
    MeshTransformerTrainer,
    MeshAutoencoderTrainer,
    MeshAutoencoder,
    MeshTransformer,MeshDataset
)
from meshgpt_pytorch.data import ( 
    derive_face_edges_from_faces
)   



  Cloning https://github.com/MarcusLoppe/meshgpt-pytorch.git to c:\users\ernest.lee\appdata\local\temp\pip-req-build-uf4kdnee
  Resolved https://github.com/MarcusLoppe/meshgpt-pytorch.git to commit b5e74674973de4a97431561d7723d9b29629a695
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/MarcusLoppe/meshgpt-pytorch.git 'C:\Users\ernest.lee\AppData\Local\Temp\pip-req-build-uf4kdnee'


Note: you may need to restart the kernel to use updated packages.


In [19]:

pkg = torch.load("./16k_autoencoder_229M_0.338.pt", map_location=torch.device("cuda")) 
autoencoder = MeshAutoencoder( 
        decoder_dims_through_depth =  (128,) * 6 + (192,) * 12 + (256,) * 24 + (384,) * 6,    
        dim_codebook = 192,  
        dim_area_embed = 16,
        dim_coor_embed = 16, 
        dim_normal_embed = 16,
        dim_angle_embed = 8,
    
    attn_decoder_depth  = 4,
    attn_encoder_depth = 2
    ).to("cuda")
autoencoder.load_state_dict(pkg['model'])

transformer = MeshTransformer(
    autoencoder,
    dim = 768,
    coarse_pre_gateloop_depth =2,  
    fine_pre_gateloop_depth= 2, 
    attn_depth = 12,  
    attn_heads = 12, 
    fine_cross_attend_text = True,
    text_cond_with_film = False,
    cross_attn_num_mem_kv = 4,
    num_sos_tokens = 1, 
    dropout  = 0.0,
    max_seq_len = 1500, 
    fine_attn_depth = 2,
    condition_on_text = True, 
    gateloop_use_heinsen = False,
    text_condition_model_types = "bge", 
    text_condition_cond_drop_prob = 0.0, 
).to("cuda")

c:\Users\ernest.lee\scoop\apps\python\current\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [20]:
%cd C:/Users/ernest.lee/OneDrive/Desktop/text-to-mesh

C:\Users\ernest.lee\OneDrive\Desktop\text-to-mesh


C:\Users\ernest.lee\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
datasets = [
    "objverse_250f_45.9M_3086_labels_53730_10_min_x1_aug.npz",
    "objverse_250f_229.7M_3086_labels_268650_10_min_x5_aug.npz"
]

dataset = None

for ds in datasets:
    if dataset is None:
        dataset = MeshDataset.load(ds)
    else:
        temp_dataset = MeshDataset.load(ds)
        dataset.data.extend(temp_dataset.data)

dataset.sort_dataset_keys()

[MeshDataset] Loaded 53730 entrys
[MeshDataset] Created from 53730 entrys
[MeshDataset] Loaded 268650 entrys
[MeshDataset] Created from 268650 entrys


In [23]:
batch_size = 16
grad_accum_every = 4

for rate in [1e-3, 1e-4]:
    trainer = MeshTransformerTrainer(model = transformer,warmup_steps = 10,grad_accum_every=grad_accum_every,num_train_steps=100, dataset = dataset, 
                                    learning_rate = rate, batch_size=batch_size ,checkpoint_every_epoch = 25) 
loss = trainer.train(35, stop_at_loss = 0.00005)   

Epoch 1/35:   0%|          | 0/20148 [00:00<?, ?it/s]c:\Users\ernest.lee\scoop\apps\python\current\Lib\site-packages\torch\backends\cuda\__init__.py:342: FutureWarning: torch.backends.cuda.sdp_kernel() is deprecated. In the future, this context manager will be removed. Please see, torch.nn.attention.sdpa_kernel() for the new context manager, with updated signature.
  warnings.warn(
Epoch 1/35:   0%|          | 52/20148 [03:42<19:40:23,  3.52s/it, loss=9.64]

In [ ]:
pkg = dict( model = transformer.state_dict(), ) 
torch.save(pkg, str("./MeshGPT-transformer_trained.pt"))